<a href="https://colab.research.google.com/github/Farhad-Davaripour/Navigating_GenAI/blob/main/rag/basic_rag_demo.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
# Install necessary libraries
%pip install openai numpy scikit-learn PyPDF2
%pip install openai openai

In [3]:
import os
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2

OPENAI_API_KEY=""

client = OpenAI(
  api_key=OPENAI_API_KEY
)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
from google.colab import files
import shutil

# Specify the folder path in your Google Drive where you want to save the file
folder_path = '/content/drive/My Drive/Navigating_GenAI_RAG/'  # Change 'your_folder_name' to your target folder name

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# URL of the file to be downloaded
file_url = 'https://raw.githubusercontent.com/Farhad-Davaripour/Navigating_GenAI/main/docs/made_up_terms.pdf'

# Download the file using wget
!wget -P /content/ $file_url

# Extract the file name from the URL
file_name = file_url.split('/')[-1]

# Move the downloaded file to the specified folder in Google Drive
shutil.move(f'/content/{file_name}', folder_path + file_name)

print(f"File downloaded and moved to {folder_path}")

--2024-05-21 11:08:53--  https://raw.githubusercontent.com/Farhad-Davaripour/Navigating_GenAI/main/docs/made_up_terms.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2646 (2.6K) [application/octet-stream]
Saving to: ‘/content/made_up_terms.pdf’

made_up_terms.pdf   100%[===================>]   2.58K  --.-KB/s    in 0s      

2024-05-21 11:08:53 (34.0 MB/s) - ‘/content/made_up_terms.pdf’ saved [2646/2646]

File downloaded and moved to /content/drive/My Drive/Navigating_GenAI_RAG/


In [23]:
# Function to read text from a PDF
def read_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Load text from a PDF
pdf_text = read_pdf(folder_path + file_name)

In [24]:
print(f"Text length:\n{len(pdf_text)}\n")
print(f"First 100 char:\n{pdf_text[:100]}")

Text length:
1952

First 100 char:
Quarintium Drift in Quantum Computing
In the field of quantum computing, researchers have recently d


In [25]:
# Function to split text into chunks with overlap
def split_text(text, chunk_size=400, overlap=20):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        if len(chunk) > 0:
            chunks.append(chunk)
    return chunks

# Split text into chunks
chunks = split_text(pdf_text)

In [26]:
print(f"Chunks length:\n{len(chunks)}\n")
print(f"First chunk:\n{chunks[0]}")

Chunks length:
6

First chunk:
Quarintium Drift in Quantum Computing
In the field of quantum computing, researchers have recently discovered a phenomenon they call
Quarintium Drift. This phenomenon occurs when particles within a quantum system experience an
unexpected shift in their state, resulting in unpredictable outcomes during computation. The
Quarintium Drift has perplexed scientists due to its seemingly random nature, ye


In [27]:
# Function to create embeddings using OpenAI API
def create_embeddings(chunks):
    embeddings = []
    for chunk in chunks:
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-ada-002"  # Choose the appropriate embedding model
        )
        embeddings.append(response.data[0].embedding)
    return np.array(embeddings)

# Create embeddings for the chunks
chunk_embeddings = create_embeddings(chunks)

In [28]:
# Function to perform vector search
def vector_search(query, chunk_embeddings, chunks, top_k):
    query_embedding = create_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], chunk_embeddings)[0]
    top_k_indices = similarities.argsort()[-top_k:][::-1]
    return [chunks[i] for i in top_k_indices]

# Example query
query = "what is Quarintium Drift in Quantum Computing?"

# Perform vector search
top_k = 1
relevant_chunks = vector_search(query, chunk_embeddings, chunks, top_k)

In [29]:
# Generate response using the language model
context = "\n".join(relevant_chunks)
prompt=[
    {
    "role": "system",
    "content": [
        {
        "type": "text",
        "text": "You are an AI assistant that provides answer to the user's query solely based on the provided content."
        }
    ]
    },
    {
    "role": "user",
    "content": [
        {
        "type": "text",
        "text": f"Based on the following information:\n{context}\n\nAnswer the following question:\n{query}"
        }
    ]
    },
]
print(f"Prompt:\n{prompt}\n")

Prompt:
[{'role': 'system', 'content': [{'type': 'text', 'text': "You are an AI assistant that provides answer to the user's query solely based on the provided content."}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Based on the following information:\nQuarintium Drift in Quantum Computing\nIn the field of quantum computing, researchers have recently discovered a phenomenon they call\nQuarintium Drift. This phenomenon occurs when particles within a quantum system experience an\nunexpected shift in their state, resulting in unpredictable outcomes during computation. The\nQuarintium Drift has perplexed scientists due to its seemingly random nature, ye\n\nAnswer the following question:\nwhat is Quarintium Drift in Quantum Computing?'}]}]



In [30]:
response = client.chat.completions.create(
    model="gpt-4o",  # Replace with the appropriate model name
    messages=prompt,
    max_tokens=150
)

In [35]:
def format_text(text, max_words_per_line=21):
    words = text.split()
    lines = []
    for i in range(0, len(words), max_words_per_line):
        line = " ".join(words[i:i + max_words_per_line])
        lines.append(line)
    return "\n".join(lines)

response_text = response.choices[0].message.content
print(format_text(response_text))

Quarintium Drift in quantum computing is a recently discovered phenomenon where particles within a quantum system experience an unexpected shift in
their state. This shift results in unpredictable outcomes during computation, and the random nature of this drift has perplexed scientists.
